In [26]:
from pathlib import Path
import json
import re
from tqdm import tqdm

settings = json.loads(Path('../../../settings.json').read_text())preprocessed_data_path = Path(settings['preprocessed_data_path'])data_path = Path('.').resolve()data_name = data_path.namesource_name = data_path.parent.namepreprocessed_dir = preprocessed_data_path/source_name/data_namesource_data_dir = preprocessed_dir/'preprocessed'
splits = ['train', 'valid']
tasks = ['LM(한국어)', '제목 짓기', '문서 생성 요약', '문서 추출 요약']
task_data_dir = preprocessed_dir/'preprocessed_task'
task_data_dir.mkdir(exist_ok=True)
for task in tasks:
    task_path = task_data_dir/task
    task_path.mkdir(exist_ok=True)
    
#### prepare for task preprocess



#### prepare for task preprocess end

#### task preprocess
for split in splits:
    source_data_dir_split = source_data_dir/split
    task_files = [(task_data_dir/task/f'{split}.jsonl').open('w', encoding='utf-8') for task in tasks]
    for source_data in tqdm(list(source_data_dir_split.iterdir()), desc=split):
        source_data = source_data.open()
        for line in source_data.readlines():
            line = json.loads(line)
            
            #### data preprocess
            doc_id = line['Meta(Acqusition)']['doc_id']
            doc_name = line['Meta(Acqusition)']['doc_name']
            passage = line['Meta(Refine)']['passage']
            passage_id = line['Meta(Refine)']['passage_id']
            abstractive_summary = line['Annotation']['summary1']
            extractive_summary = line['Annotation']['summary2']
            publisher = line['Meta(Acqusition)']['publisher']
            publisher_year = line['Meta(Acqusition)']['publisher_year']
            if extractive_summary == None:
                extractive_summary = line['Annotation']['summary3']
            
            
            #### data preprocess end 

            #### LM(한국어)
            data = {'text': None}
            ## preprocess data from line
            text = f'publisher : {publisher} {publisher_year}\ntitle : {doc_name}\npassage : {passage}\n'
            data['text'] = text
            ## preprocess data from line end
            task_files[0].write(json.dumps(data, ensure_ascii=False)+'\n')
            #### LM(한국어) end


            #### 제목 짓기
            data = {'input': {}, 'output': None}
            ## preprocess data from line
            data['input']['doc_id'] = doc_id
            data['input']['passage_id'] = passage_id            
            data['input']['publisher'] = publisher
            data['input']['publisher_year'] = publisher_year
            data['input']['passage'] = passage
            data['output'] = doc_name
            ## preprocess data from line end
            task_files[1].write(json.dumps(data, ensure_ascii=False)+'\n')
            # print(json.dumps(data, indent=4, ensure_ascii=False))
            #### 제목 짓기 end


            #### 문서 생성 요약
            ## preprocess data from line
            data['input']['title'] = doc_name
            data['input']['passage'] = data['input'].pop('passage')
            data['output'] = abstractive_summary
            ## preprocess data from line end
            task_files[2].write(json.dumps(data, ensure_ascii=False)+'\n')
            # print(json.dumps(data, indent=4, ensure_ascii=False))
            #### 문서 생성 요약 end


            #### 문서 추출 요약
            ## preprocess data from line
            data['output'] = extractive_summary
            ## preprocess data from line end
            task_files[3].write(json.dumps(data, ensure_ascii=False)+'\n')
            # print(json.dumps(data, indent=4, ensure_ascii=False))
            #### 문서 추출 요약 end


            
    #         break
    #     break
    # break

    for path in task_files:
        path.close()      
        
#### task preprocess end      

valid: 100%|██████████| 1/1 [00:01<00:00,  1.33s/it]
